In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import os
import plotly.io as plt
from pretty_html_table import build_table
# Enable this to export as iframe
# plt_io.renderers.default = 'iframe'
pd.options.mode.chained_assignment = None

In [ ]:
# Global variables
default_font_family = 'IBM Plex Sans'
default_font_size = 12
colors = ['#E7C65B', '#225560', '#310D20', '#96031A']
logo_path = os.path.join(os.getcwd(), 'INPUT/sip-logo.png')
DEBUG = True

# 1. Data Preprocessing

In [ ]:
trello_table = pd.read_csv('./INPUT/trello_board.csv')
trello_fields_dict = {
    'Card Name': 'T#',
    'Card Description': 'DESC',
    'Labels': 'LABELS',
    'List Name': 'STATUS',
    'T_CRE_TS': 'TICKET_CREATION_TIMESTAMP',
    'Card ID': 'TICKET_RESPONSE_TIMESTAMP',
    'T_RES_TS': 'TICKET_RESOLUTION_TIMESTAMP',
    'A#': 'ALERT_ID',
    'O#': 'OFFENSE_ID'
    }
trello_table = trello_table[trello_fields_dict]
trello_table.rename(columns=trello_fields_dict, inplace=True)

# Only analyze resolved tickets
trello_table = trello_table[trello_table['STATUS'] == 'RESOLVED_VSOC']
trello_table = trello_table.sort_values(by=['T#'], ignore_index=True)

if DEBUG:
    trello_table[trello_table.isna().any(axis=1)]

# Convert timestamps datetime objects
trello_table['TICKET_CREATION_TIMESTAMP'] = \
    pd.to_datetime(trello_table['TICKET_CREATION_TIMESTAMP'])
trello_table['TICKET_RESOLUTION_TIMESTAMP'] = \
    pd.to_datetime(trello_table['TICKET_RESOLUTION_TIMESTAMP'])

# Generate start and end date of the trello table
trello_table.start_timestamp = trello_table['TICKET_CREATION_TIMESTAMP'
        ].min().strftime('%d/%m/%Y|%H:%M:%S')
trello_table.end_timestamp = trello_table['TICKET_CREATION_TIMESTAMP'
        ].max().strftime('%d/%m/%Y|%H:%M:%S')
trello_table.no_days = len(trello_table['TICKET_CREATION_TIMESTAMP'
                           ].dt.normalize().unique())


In [ ]:
# To get the card creation timestamp from it's ID
trello_table['TICKET_RESPONSE_TIMESTAMP'] = [x[:8] for x in trello_table['TICKET_RESPONSE_TIMESTAMP']]
trello_table['TICKET_RESPONSE_TIMESTAMP'] = trello_table['TICKET_RESPONSE_TIMESTAMP'].apply(int, base=16)
trello_table['TICKET_RESPONSE_TIMESTAMP'] = pd.to_datetime(trello_table['TICKET_RESPONSE_TIMESTAMP'],unit='s')
trello_table['TICKET_RESPONSE_TIMESTAMP'] = trello_table['TICKET_RESPONSE_TIMESTAMP'].dt.tz_localize('GMT').dt.tz_convert('Asia/Riyadh').dt.tz_localize(None)

trello_table['TICKET_CREATION_TIMESTAMP_ADJUSTED'] = trello_table['TICKET_CREATION_TIMESTAMP']
trello_table['TICKET_RESPONSE_TIMESTAMP_ADJUSTED'] = trello_table['TICKET_RESPONSE_TIMESTAMP']
trello_table['TICKET_RESOLUTION_TIMESTAMP_ADJUSTED'] = trello_table['TICKET_RESOLUTION_TIMESTAMP']
if DEBUG:
    display(trello_table[trello_table['TICKET_CREATION_TIMESTAMP'].isnull()])
    display(trello_table[trello_table['TICKET_RESOLUTION_TIMESTAMP'].isnull()])    # Check if we have error in timestamps
    trello_table['ERROR'] = (trello_table['TICKET_CREATION_TIMESTAMP'
                             ].dt.second
                             >= trello_table['TICKET_RESOLUTION_TIMESTAMP'
                             ].dt.second).astype(int)
    
display(trello_table[trello_table.duplicated('T#')])
trello_table.drop_duplicates(subset ="T#",keep = False, inplace = True)

In [ ]:
# For monthly report
trello_table = trello_table[(trello_table['TICKET_CREATION_TIMESTAMP'] >= '2022-05-01 00:00:00')]
trello_table = trello_table[(trello_table['TICKET_CREATION_TIMESTAMP'] < '2022-11-01 00:00:00')]
# print(len(trello_table))
# display(trello_table)

# For weekly report
# trello_table = trello_table[(trello_table['TICKET_RESOLUTION_TIMESTAMP'] >= '2022-10-20 16:00:00')]
# trello_table = trello_table[(trello_table['TICKET_RESOLUTION_TIMESTAMP'] <= '2022-10-27 15:59:59')]
trello_table.drop_duplicates(subset ="T#",keep = False, inplace = True)
print(len(trello_table))
# display(trello_table)

In [ ]:
sip_bh = pd.offsets.CustomBusinessHour(start='08:15', end='15:30', weekmask='Sun Mon Tue Wed Thu')

for index, row in trello_table.iterrows():
    roll1 = sip_bh.rollforward(pd.Timestamp(trello_table['TICKET_CREATION_TIMESTAMP'][index]))
    trello_table['TICKET_CREATION_TIMESTAMP_ADJUSTED'][index] = roll1
    roll2 = sip_bh.rollforward(pd.Timestamp(trello_table['TICKET_RESPONSE_TIMESTAMP'][index]))
    trello_table['TICKET_RESPONSE_TIMESTAMP_ADJUSTED'][index] = roll2
    roll3 = sip_bh.rollforward(pd.Timestamp(trello_table['TICKET_RESOLUTION_TIMESTAMP'][index]))
    trello_table['TICKET_RESOLUTION_TIMESTAMP_ADJUSTED'][index] = roll3


trello_table['BUSINSESS_HOURS_TO_ACKNOWLEDGE'] = trello_table.apply(lambda x: len(pd.date_range(start=x.TICKET_CREATION_TIMESTAMP, end=x.TICKET_RESPONSE_TIMESTAMP, freq= sip_bh)),axis=1)
trello_table['BUSINSESS_HOURS_TO_RESOLVE'] = trello_table.apply(lambda x: len(pd.date_range(start=x.TICKET_CREATION_TIMESTAMP, end=x.TICKET_RESOLUTION_TIMESTAMP, freq= sip_bh)),axis=1)

# TICKETS/BUSINESS HOURS ratio
tickets_business_hours_ratio = len(trello_table)/len(pd.bdate_range(start=trello_table['TICKET_CREATION_TIMESTAMP'].min(), end=trello_table['TICKET_CREATION_TIMESTAMP'].max(), freq= sip_bh))

print('Rate of resolved VSOC ticket per business hour = ', tickets_business_hours_ratio, 'tickets per business hour')

In [ ]:
# Handle labels
labels_dict = {'SC' : 'SEC_CONTROL', 'RC' : 'RES_CODE', 'CO' : 'DEPENDENT_ON', 'TB' : 'ROOTICKET_CAUSE', 'PR' : 'PRIORITY'}
trello_table["LABELS"] = trello_table["LABELS"].str.replace('\([^)]*\)', '', regex=True)
trello_table["LABELS"] = trello_table["LABELS"].str.replace(',', ' ', regex=True)
#  Temp. solution :)
trello_table['SEC_CONTROL'] = trello_table['LABELS'].str.extract(r'(\bSC\d{2}\b)')
trello_table['RES_CODE'] = trello_table['LABELS'].str.extract(r'(\bRC\d{2}\b)')
trello_table['DEPENDENT_ON'] = trello_table['LABELS'].str.extract(r'(\bCO\d{2}\b)')
trello_table['ROOT_CAUSE'] = trello_table['LABELS'].str.extract(r'(\bTB\d{2}\b)')
trello_table['PRIORITY'] = trello_table['LABELS'].str.extract(r'(\bPR\d{2}\b)')
# Load labels translation as a separate external file to keep the confidentiality of your log sources
labels_translation = pd.read_csv('./INPUT/labels_translation.csv', index_col=0, header=None, squeeze=True).to_dict()
# print(labels_translation)
trello_table = trello_table.replace(labels_translation)
trello_table.drop('LABELS', axis=1, inplace=True)


In [ ]:
# mtta = round(trello_table['BUSINSESS_HOURS_TO_ACKNOWLEDGE'].mean(), 2) 
# mttr = round(trello_table['BUSINSESS_HOURS_TO_RESOLVE'].mean(), 2) 
# if DEBUG:    
    # print('Mean time to resolve = ', mttr, 'business hours')
    # print('Mean time to ack = ', mtta, 'business hours')

# 2. Data Visualization 

In [ ]:
required_fields = ['SEC_CONTROL', 'RES_CODE']

trello_table.start_timestamp = trello_table['TICKET_CREATION_TIMESTAMP'].min().strftime('%d/%m/%Y|%H:%M:%S')
trello_table.end_timestamp = trello_table['TICKET_CREATION_TIMESTAMP'].max().strftime('%d/%m/%Y|%H:%M:%S')
trello_table.no_days = len(trello_table['TICKET_CREATION_TIMESTAMP'].dt.normalize().unique())

for required_field in required_fields:
    copy_of_trello_table = trello_table.copy()
    series = pd.value_counts(copy_of_trello_table[required_field])
    mask = (series/series.sum() * 100).lt(1.0)
    # copy_of_trello_table[required_field] = np.where(copy_of_trello_table[required_field].isin(series[mask].index),'OTHERS ≤ 1.0%'.format(required_field),copy_of_trello_table[required_field])
    required_field_count = copy_of_trello_table[required_field].value_counts()    
    required_field_count = required_field_count.rename_axis(required_field).reset_index(name='{}_COUNT'.format(required_field))
    required_field_count = required_field_count.reset_index(drop=True)
    required_field_count.index.rename('NO.', inplace=True)
    required_field_count.index+=1
    required_field_count['{}_PCT'.format(required_field)] = required_field_count['{}_COUNT'.format(required_field)] / required_field_count['{}_COUNT'.format(required_field)].sum()
    required_field_count.to_csv('./OUTPUT/{}.csv'.format(required_field), sep=',')
    fig = go.Figure(data=[go.Pie(labels=required_field_count[required_field], values=required_field_count['{}_PCT'.format(required_field)],textinfo='label+percent',
                             insidetextorientation='radial', showlegend=False,marker=dict(colors=colors, line=dict(color='#000000', width=2))
                            )])
    fig.update_layout(font_family=default_font_family)
    fig.update_layout(title='Security Investigation Tickets distributed by<br> {}'.format(required_field))
    fig.layout.images = [dict( source=logo_path, xref='paper', yref='paper', x=0.97,  y=0.97, sizex=0.50, sizey=0.50, xanchor='center', yanchor='bottom')]
    fig.write_image("./OUTPUT/{}_COUNT.svg".format(required_field))

    # fig.show()

In [ ]:
# Generate a summary table of the main features
duration_start = trello_table['TICKET_CREATION_TIMESTAMP'].min().strftime('%Y-%m-%d')
print(duration_start)
summary_table_count = trello_table.groupby(['RES_CODE']).size()
summary_table_count.to_excel('./OUTPUT/SUMMARY_TABLE_COUNT.xlsx'.format(duration_start))


In [ ]:
trello_table.start_timestamp = trello_table['TICKET_CREATION_TIMESTAMP'
        ].min().strftime('%d/%m/%Y|%H:%M:%S')
trello_table.end_timestamp = trello_table['TICKET_CREATION_TIMESTAMP'
        ].max().strftime('%d/%m/%Y|%H:%M:%S')
trello_table.no_days = len(trello_table['TICKET_CREATION_TIMESTAMP'
                           ].dt.normalize().unique())

tickets_count = trello_table['T#'].value_counts()
tickets_count = tickets_count.rename_axis('T#'
        ).reset_index(name='T#_COUNT')
tickets_count = tickets_count.reset_index(drop=True)
tickets_count.index.rename('NO.', inplace=True)
tickets_count.index += 1
tickets_count['T#_PCT'] = tickets_count['T#_COUNT'] \
    / tickets_count['T#_COUNT'].sum()
s = pd.to_datetime(trello_table['TICKET_CREATION_TIMESTAMP'])
tickets_count = s.groupby(s.dt.floor('d'
                          )).size().reset_index(name='COUNT')

# Plot ----------------------------------------------------------------------------------------------

my_layout = go.Layout(title=default_font_family, font=dict(color='#7f7f7f',
                      size=default_font_size))
my_data = go.Scatter(x=tickets_count['TICKET_CREATION_TIMESTAMP'],
                     y=tickets_count['COUNT'], mode='lines+markers+text'
                     )
fig = go.Figure(data=my_data)

fig.update_layout(shapes=[go.layout.Shape(  # Line Horizontal
    type='line',
    x0=tickets_count['TICKET_CREATION_TIMESTAMP'].min(),
    y0=tickets_count['COUNT'].mean(),
    x1=tickets_count['TICKET_CREATION_TIMESTAMP'].max(),
    y1=tickets_count['COUNT'].mean(),
    line=dict(color='black', width=1, dash='longdash'),
    )])

fig.add_trace(go.Scatter(
    x=[tickets_count['TICKET_CREATION_TIMESTAMP'].max()
       - pd.Timedelta(days=trello_table.no_days) / 2.0],
    y=[tickets_count['COUNT'].mean()],
    mode='markers+text',
    name='Markers and Text',
    hoverinfo='skip',
    textposition='top right',
    ))

fig.update_traces(marker_color='rgb(231,198,91)',
                  marker_line_color='black', marker_line_width=1,
                  opacity=1.0)
fig.update_layout(title_text='VSOC tickets trendline grouped by the day<br>Dashed line represents the average no. VSOC tickets ({})'.format(int(tickets_count['COUNT'
                  ].mean())), font_size=default_font_size)

fig.update_layout(font_family=default_font_family, showlegend=False)
fig.layout.images = [dict(
    source=logo_path,
    x=0.9,
    y=1.05,
    sizex=0.25,
    sizey=0.25,
    xanchor='center',
    yanchor='bottom',
    )]
fig.update_layout(font_size=default_font_size)
fig.write_image('./OUTPUT/TRENDLINE.svg')


In [ ]:
#  FIGURE 1: EOI Chart
res_codes = ['Non-Issue', 'False Positive','True Positive']
last_week_vsoc_tickets_count = [20, 0, 1]
current_week_vsoc_tickets_count = [35, 0, 1]
ce_sum = list(((np.array(last_week_vsoc_tickets_count ) + np.array(current_week_vsoc_tickets_count))))
delta_ce = list(((np.array(current_week_vsoc_tickets_count) - np.array(last_week_vsoc_tickets_count )) / last_week_vsoc_tickets_count ) * 100)
delta_ce = np.round(delta_ce, decimals=1)
delta_ce = np.nan_to_num(delta_ce)

green_tag = '<span style="color:green">▼</span>'
red_tag = '<span style="color:red">▲</span>'

title_plot1 = 'Increase {} and Decrease {} in VSOC Tickets Count Sorted by Resolution Code'.format(red_tag, green_tag)


fig = go.Figure()


fig.layout.images = [dict(
    source=logo_path,
    xref='paper', 
    yref='paper',
    x=0.05, 
    y=1.05,
    sizex=0.15, 
    sizey=0.15,
    xanchor='center', 
    yanchor='bottom')]

fig.add_trace(go.Bar(
    x=res_codes,
    y=last_week_vsoc_tickets_count,
    name='Resolved from 04 August 04:00pm to 11 August 03:59pm',
    marker_color='#4E4E4E', 
    marker_line_color='black',
    offset = 0,
    marker_line_width=1.5, 
    opacity=0.7, width=0.6))

fig.add_trace(go.Bar(
    x=res_codes,
    y=current_week_vsoc_tickets_count,
    name='Resolved from 11 August 04:00pm to 18 August 03:59pm', 
    marker_color='#E7C65B', 
    marker_line_color='black',
    marker_line_width=1.5, 
    opacity=0.7, width=0.6))


fig.add_trace(go.Bar(
    x=res_codes, 
    y=max(last_week_vsoc_tickets_count,current_week_vsoc_tickets_count),
    offset=0,
    marker_line_width=1.5, 
    opacity=0, 
    width=0.8,
    showlegend=False))



fig.update_layout(
    height=600,
    width=900,
    template='plotly_white',
    font_family=default_font_family,
    font_size=default_font_size,
    barmode='overlay', 
    xaxis_tickangle=0,
    title=title_plot1,
    title_x=0.5, 
    font=dict(color='black'),
    legend=dict(
    orientation='h',
    yanchor='top',
    y=1.1,
    xanchor='center',
    x=0.5))

# for i, (x, y, z) in enumerate(zip(last_week_vsoc_tickets_count, current_week_vsoc_tickets_count, delta_ce)):
#     print(i, x, y, z)
#     if z > 0:
#         fig.add_annotation(x=i,y=max(x-4, y+2),text='Increased from {} to {} (▲+{}%)'.format(x, y, z),showarrow=False,font=dict(color='red', size=12),align='center')
#     else:
#         fig.add_annotation(x=i,y=max(x-4, y+2),text='Decreased from {} to {} (▼{}%)'.format(x,y,z),showarrow=False,font=dict(color='green', size=12),align='center')



for i, (x, y, z) in enumerate(zip(last_week_vsoc_tickets_count, current_week_vsoc_tickets_count, delta_ce)):
    print(i, x, y, z)
    
    if z > 0:
        arrow = '▲'
        arrow_color = 'red'
        direction = 'Increased'
        sign = '+'
    elif z < 0:
        arrow = '▼'
        arrow_color = 'green'
        direction = 'Decreased'
        sign = ''

    elif z == 0:
        arrow = ''
        arrow_color = 'gray'
        direction = 'Stayed the same'
        sign = '' 

    # fig.add_annotation(x=i,y=max(x, y),text='{} from {} to {} ({}{}{}%)'.format(direction,x, y, arrow,sign,z),showarrow=True,font=dict(color=arrow_color, size=12),align='right')
    fig.add_annotation(x=i,y=max(x, y),text='{} from {} to {}'.format(direction,x, y),showarrow=True,font=dict(color=arrow_color, size=12),align='right')

fig.write_image('./OUTPUT/EOI.svg')
fig.show()


# 3. Reports Generation 

In [ ]:
duration_start = trello_table['TICKET_RESOLUTION_TIMESTAMP'].min().strftime('%Y-%m-%d')
duration_end = trello_table['TICKET_RESOLUTION_TIMESTAMP'].max().strftime('%Y-%m-%d')


# Save an analyst complete report
trello_table = trello_table.reset_index(drop=True)
trello_table.index+=1
file_name = '{}-{} SOC_REPORT.html'.format(duration_start, duration_end)
trello_table.to_html('./OUTPUT/{}'.format(file_name))
# trello_table.to_excel('./OUTPUT/report.xlsx')
html_table_blue_light = build_table(trello_table, 'grey_light', font_family=default_font_family, index=True)
with open('./OUTPUT/{}'.format(file_name), 'w') as f:
    f.write(html_table_blue_light)


# Save a customer report
customer_report = trello_table[['T#','TICKET_CREATION_TIMESTAMP','OFFENSE_ID','SEC_CONTROL','ALERT_ID', 'RES_CODE', 'DESC', 'PRIORITY']]
customer_report = customer_report.sort_values(by=['T#'], ignore_index=True)
file_name = 'CUS_SOC_REPORT.xlsx'
customer_report.to_excel('./OUTPUT/{}'.format(file_name))
# html_table_blue_light = build_table(customer_report, 'grey_light', font_family=default_font_family, index=True)
# with open('./OUTPUT/{}'.format(file_name), 'w') as f:
    # f.write(html_table_blue_light)

# 4. Test Bench

In [ ]:
# import numpy as np
# import scipy
# import matplotlib
# import matplotlib.pyplot as plt
# from scipy import stats
# mydata = np.random.randn(26)
# mymean = np.mean(mydata)
# mystd = np.std(mydata)
# mypd = scipy.stats.norm.pdf((mydata), mymean, mystd)
# plt.plot(mydata, mypd)

In [ ]:
# import plotly.figure_factory as ff
# import numpy as np

# # Add histogram data
# x1 = np.random.randn(200) - 2
# x2 = np.random.randn(200)
# x3 = np.random.randn(200) + 2
# x4 = np.random.randn(200) + 4

# # Group data together
# hist_data = [x1, x2, x3, x4]

# group_labels = ['Group 1', 'Group 2', 'Group 3', 'Group 4']

# # Create distplot with custom bin_size
# fig = ff.create_distplot(hist_data, group_labels, bin_size=.2)
# fig.show()

In [ ]:
# Not needed anymore
# mbhtr_by_department = pd.DataFrame(columns = ['DEPENDENT_ON','MEAN_BUSINSESS_HOURS_TO_RESOLVE'])

# for dept in trello_table['DEPENDENT_ON'].unique():
#     test1 = trello_table[trello_table['DEPENDENT_ON'].str.contains(dept)]
#     mbhtr_by_department = mbhtr_by_department.append({'DEPENDENT_ON': dept, 'MEAN_BUSINSESS_HOURS_TO_RESOLVE': round(test1['BUSINSESS_HOURS_TO_RESOLVE'].mean(),2)}, ignore_index=True)




# mbhta_by_department = pd.DataFrame(columns = ['DEPENDENT_ON','MEAN_BUSINSESS_HOURS_TO_ACK'])

# for dept in trello_table['DEPENDENT_ON'].unique():
#     test2 = trello_table[trello_table['DEPENDENT_ON'].str.contains(dept)]
#     mbhta_by_department = mbhta_by_department.append({'DEPENDENT_ON': dept, 'MEAN_BUSINSESS_HOURS_TO_ACK': round(test2['BUSINSESS_HOURS_TO_ACKNOWLEDGE'].mean(),2)}, ignore_index=True)



In [ ]:
# mbhta_by_priority = pd.DataFrame(columns = ['PRIORITY','MEAN_BUSINSESS_HOURS_TO_ACK'])


# for prio in trello_table['PRIORITY'].unique():
#     test = trello_table[trello_table['PRIORITY'].str.contains(prio)]
#     mbhtr_by_priority = mbhtr_by_priority.append({'PRIORITY': prio, 'MEAN_BUSINSESS_HOURS_TO_ACK': round(test['BUSINSESS_HOURS_TO_ACKNOWLEDGE'].mean(),2)}, ignore_index=True)


# fig = px.bar(mbhtr_by_priority, x='PRIORITY', y='MEAN_BUSINSESS_HOURS_TO_ACK', text='MEAN_BUSINSESS_HOURS_TO_ACK')
# fig.update_traces(marker_color='#E7C65B', marker_line_color='black',
#                   marker_line_width=1.5, opacity=1)
# fig.layout.images = [dict( source=logo_path, xref='paper', yref='paper', x=0.95,  y=1, sizex=0.25, sizey=0.25, xanchor='center', yanchor='bottom')]

# fig.update_layout(
#     title="Mean Business Hours to ACK By Priority",
#     xaxis_title='Ticket Priority',
#     yaxis_title='Total number of business hours',
#     font=dict(
#         family=default_font_family,
#         size=default_font_size,
#     )
# )
# fig.write_image('./OUTPUT/MEAN_BUSINSESS_HOURS_TO_ACK_BY_PRIORITY.svg')

In [ ]:
# Not needed anymore
# fig = px.bar(mbhtr_by_department, x='DEPENDENT_ON', y='MEAN_BUSINSESS_HOURS_TO_RESOLVE', text='MEAN_BUSINSESS_HOURS_TO_RESOLVE')
# fig.update_traces(marker_color='#E7C65B', marker_line_color='black',
#                   marker_line_width=1.5, opacity=1)
# fig.layout.images = [dict( source=logo_path, xref='paper', yref='paper', x=0.95,  y=1, sizex=0.25, sizey=0.25, xanchor='center', yanchor='bottom')]

# fig.update_layout(
#     title="Mean Business Hours to Resolve By Department",
#     xaxis_title='Dependency by department',
#     yaxis_title='Total number of business hours',
#     font=dict(
#         family=default_font_family,
#         size=default_font_size,
#     )
# )
# fig.write_image('./OUTPUT/MEAN_BUSINSESS_HOURS_TO_RESOLVE_BY_DEPARTMENT.svg')

In [ ]:
# mbhtr_by_priority = pd.DataFrame(columns = ['PRIORITY','MEAN_BUSINSESS_HOURS_TO_RESOLVE'])



# for prio in trello_table['PRIORITY'].unique():
#     test = trello_table[trello_table['PRIORITY'].str.contains(prio)]
#     mbhtr_by_priority = mbhtr_by_priority.append({'PRIORITY': prio, 'MEAN_BUSINSESS_HOURS_TO_RESOLVE': round(test['BUSINSESS_HOURS_TO_RESOLVE'].mean(),2)}, ignore_index=True)


# fig = px.bar(mbhtr_by_priority, x='PRIORITY', y='MEAN_BUSINSESS_HOURS_TO_RESOLVE', text='MEAN_BUSINSESS_HOURS_TO_RESOLVE')
# fig.update_traces(marker_color='#E7C65B', marker_line_color='black',
#                   marker_line_width=1.5, opacity=1)
# fig.layout.images = [dict( source=logo_path, xref='paper', yref='paper', x=0.95,  y=1, sizex=0.25, sizey=0.25, xanchor='center', yanchor='bottom')]

# fig.update_layout(
#     title="Mean Business Hours to Resolve By Priority",
#     xaxis_title='Ticket Priority',
#     yaxis_title='Total number of business hours',
#     font=dict(
#         family=default_font_family,
#         size=default_font_size,
#     )
# )
# fig.write_image('./OUTPUT/MEAN_BUSINSESS_HOURS_TO_RESOLVE_BY_PRIORITY.svg')

In [ ]:
# fig = px.sunburst(trello_table, path=['PRIORITY', 'RES_CODE'], values='BUSINSESS_HOURS_TO_ACKNOWLEDGE',
#                   color='BUSINSESS_HOURS_TO_ACKNOWLEDGE', hover_data=['BUSINSESS_HOURS_TO_ACKNOWLEDGE'],
#                   color_continuous_midpoint=np.average(trello_table['BUSINSESS_HOURS_TO_ACKNOWLEDGE'], weights=trello_table['BUSINSESS_HOURS_TO_ACKNOWLEDGE']))
# fig.show()